In [ ]:
import numpy as np
from icglm.iclamp import IClamp
ic = IClamp.load_from_abf(path='./18o03049.abf')
ic = ic.subtract_Ih(th=1000.1).restrict(t0=1000.1, tf=11000.1)
ic.plot(sweeps=[0])

In [ ]:
from icglm.glm_fitting import GLMFitter
glm_fit = GLMFitter(ic).set_mask_spikes(thr=-13).subsample(10)

tbins_kappa = np.arange(0., 220, 5)
tbins_eta = np.arange(0., 480, 10)

glm_fit.fit(tbins_kappa=tbins_kappa, tbins_eta=tbins_eta)
glm_fit.plot_filters()

In [ ]:
glm_fit.set_mask_spikes_model(trials=9)
glm_fit.psth(psth_kernel=KernelFun.gaussian_delta(delta=40))
glm_fit.plot_raster()

In [ ]:
from icglm.glm_decoding import GLMDecoder
Imu, Isd = np.mean(glm_fit.ic.stim), np.mean(np.std(glm_fit.ic.stim, 0))

I0 = np.zeros(len(glm_fit.ic.t))
decoder = GLMDecoder(glms=[glm_fit.glm], t=glm_fit.ic.t, mask_spk=[glm_fit.mask_spikes], 
                     tau=3, Imu=[Imu], Isd=[Isd], I_true=(glm_fit.ic.stim - Imu) / Isd)

decoder.estimate_MAP_I(I0, prior='OU', verbose=True)